# Importação de dependencias

In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import numpy as np

# Obter dados através da api da Info Dengue
Documentação: [Tutorial da API](https://info.dengue.mat.br/services/tutorial/Python)


In [ ]:
url = "https://info.dengue.mat.br/api/alertcity"
geocode = 2304400
disease = "dengue"
format = "csv"
ew_start = 1
ew_end = 53
ey_start = 2015
ey_end = 2024

params =(
    "&disease="
    + f"{disease}"
    + "&geocode="
    + f"{geocode}"
    + "&disease="
    + f"{disease}"
    + "&format="
    + f"{format}"
    + "&ew_start="
    + f"{ew_start}"
    + "&ew_end="
    + f"{ew_end}"
    + "&ey_start="
    + f"{ey_start}"
    + "&ey_end="
    + f"{ey_end}"
)

url_resp = "?".join([url, params])
url_resp

# Ler os dados metereologicos pelo site INMET (Instituto nacional de metereologia)
[Link](https://portal.inmet.gov.br/dadoshistoricos)

In [ ]:
df = pd.read_csv('./data/FORTALEZA_2015_2024.CSV', encoding='latin1' ,sep=";", decimal=',')

## Limpeza e tratamento de dados

### Substituindo -9999 por np.nan em todo o DataFrame



In [ ]:
df = df.replace(-9999, np.nan)

### Formatar Data e criamos uma ramificação só pegando os valores Mensal


In [ ]:
df['Data'] = pd.to_datetime(df['Data'],format='%d/%m/%Y')
df.set_index('Data', inplace=True)

#### Condições

In [ ]:
condicoes={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':"sum",
'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)':"mean",
'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)':"mean",
'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)':"mean",
'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':"mean",
'TEMPERATURA DO PONTO DE ORVALHO (°C)':"mean",
'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)':"mean",
'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)':"mean",
'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)':"mean",
}


#### Agrupar o dataframe com suas condições

In [ ]:
df1 = df1.resample('W').agg(condicoes)

#### Colunas para remover

In [ ]:
columns_to_drop_mensal_sem_vento = ['Hora UTC', 'RADIACAO GLOBAL (Kj/m²)',
                   'UMIDADE RELATIVA DO AR, HORARIA (%)',
                   'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
                   'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)']
df1 = df.copy()

#### Arredondamento

In [ ]:
for coluna in df1.columns:
  df1[coluna]=df1[coluna].round(2)

In [ ]:
df1['data_iniSE']=df1.index

### Transformar as datas em string

In [ ]:
df1['data_iniSE']=df1['data_iniSE'].astype(str)

# Obter informações sobre o DataFrame

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df1.columns

# Realizar o cruzamento dos dados e limpeza dos dados

In [ ]:
cruzamento_dados=df1.merge(dados[['data_iniSE','nivel', 'casos','umidmax', 'tempmax',]],on='data_iniSE',how='left')

In [ ]:
cruzamento_dados.dropna(inplace=True)

In [ ]:
cruzamento_dados['data_iniSE'] = pd.to_datetime(cruzamento_dados['data_iniSE'],format='%Y-%m-%d')

In [ ]:
cruzamento_dados['dia'] = cruzamento_dados['data_iniSE'].dt.day
cruzamento_dados['mes'] = cruzamento_dados['data_iniSE'].dt.month

# Obter informações do cruzamento

In [ ]:
cruzamento_dados['nivel'].value_counts()

In [ ]:
cruzamento_dados.columns

## Iniciar os testes

In [ ]:
X = cruzamento_dados[[
   'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
   'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
      'dia',
    'mes'
]]

## Criar lags dsa variáveis


In [ ]:
met_vars = [
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    'mes',
    'dia'
]
for var in met_vars:
    for lag in [1,2,3,4]:
        cruzamento_dados[f'{var}_lag{lag}'] = cruzamento_dados[var].shift(lag)

### Limpar dados vazios

In [ ]:
cruzamento_dados.dropna(inplace=True)

In [ ]:
def is_met_var(var_name):
    for met in met_vars:
        if var_name == met or var_name.startswith(met + "_lag"):
            return True
    return False
features = [c for c in cruzamento_dados.columns if is_met_var(c)]
X = cruzamento_dados[features]
y = cruzamento_dados['nivel']

In [ ]:
n = len(cruzamento_dados)
n_train = int(0.7 * n)
X_train = X.iloc[:n_train]
X_test  = X.iloc[n_train:]
y_train = y.iloc[:n_train]
y_test  = y.iloc[n_train:]

# Árvore de decição de classificação

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

In [ ]:
resultado = decision_tree.predict(X_test)

In [ ]:
metrics.recall_score(y_test, resultado, average="micro")

In [ ]:
print(classification_report(y_test, resultado))

In [ ]:
resultado

# Random Forest

In [ ]:
valores=pd.DataFrame(columns=['indice','porcentagem'])
random_forest = RandomForestClassifier(n_estimators=18)
random_forest.fit(X_train, y_train)
resultado = random_forest.predict(X_test)
resultado_porcentagem=metrics.recall_score(y_test, resultado, average="micro")
print(classification_report(y_test, resultado))
valores.loc[len(valores)] = [18, resultado_porcentagem]
print(18,metrics.recall_score(y_test, resultado, average="micro"))